## Transfer Validation

This notebook will be used to validate the transfer of the **Javascript Google Earth Engine API** code to the new **Python API version** and subsequent checking. This is necessary due to the need of IPython and it's synchronicity with Jupyter to visualize the maps generated in Google Earth Engine. This is needed to validate every step and function that is added. 

First, let's load in the needed libraries and initialize our Google Earth Engine session: 

In [1]:
import ee
import folium
import IPython.display as ip
import geemap 
import GetImageCollection as ic
import MergeChunk as mc
import Subroutines as sub
import CloudCodes as cc
ee.Initialize()

Now, lets load in our assets and get the physical properties of it for usage in our map. 

In [2]:
nhd = ee.FeatureCollection('users/mincej20/NHD_Filtered_50')
feat = ee.Feature(nhd.filterMetadata("GNIS_ID", "equals", "00970163").first())

centroid = ee.Geometry(feat.geometry().centroid())
long = centroid.coordinates().get(0).getInfo()
lat = centroid.coordinates().get(1).getInfo()

We have to make sure that it works, so we'll verify the feature and centroid: 

In [3]:
print(feat.get("GNIS_ID").getInfo())
print(feat.get("GNIS_Nm").getInfo())
print(centroid.getInfo())

00970163
Lake George
{'type': 'Point', 'coordinates': [-73.57973412405578, 43.601748078270504]}


Looks like our **GEE** is properly connected and we can access our assets, etc. 
Now, let's see about testing the GetImageCollection scripts.

In [4]:
imgs = ic.getImageCollection(feat, True)
imgs = imgs.sort("system:time_start")
print(imgs.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'BLUE', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7841, 7111], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 506085, 0, -30, 5047515]}, {'id': 'GREEN', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7841, 7111], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 506085, 0, -30, 5047515]}, {'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7841, 7111], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 506085, 0, -30, 5047515]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7841, 7111], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 506085, 0, -30, 5047515]}, {'id': 'SWIR1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7841, 7111], 'crs': 'EPSG:32618', 'crs_transform': [3

Let's set up the map to check how the images were loaded, as well as the visual parameters. 

In [5]:
viz = {
    'bands': ["RED", "GREEN", "BLUE"],
    'min': 0, 
    'max': 3000,
    'gamma': 1.4
}

Map = geemap.Map(location=[lat, long], zoom_start=10)
Map.addLayer(imgs.first(), viz, "Test", True)
Map.addLayer(feat.geometry(), {}, "Feature", True)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=False)
Map


Map(center=[43.601748078270504, -73.57973412405578], controls=(WidgetControl(options=['position'], widget=HBox…

We should now try out the MergeCentroid script to merge the centroids. 

In [6]:
merged = mc.mergeByChunk(imgs)

Since the size is good, let's visualize one of the merged images to make sure that it is working appropriately. 

In [7]:
Map.addLayer(merged.first(), viz, "Merged", True)
Map.addLayer(feat.geometry(), {}, "Feature", True)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=False)
Map

Map(center=[43.601748078270504, -73.57973412405578], controls=(WidgetControl(options=['position'], widget=HBox…

So it seems to have worked and matches up with the incomplete image from the original map! Now, we should test out the Subroutines that have been defined, like the cloud codes and water isolation methods. 

In [8]:
img = merged.first()
img = sub.noClouds(img)
Map.addLayer(img, viz, "NoCloud", True)
Map

Map(center=[43.601748078270504, -73.57973412405578], controls=(WidgetControl(options=['position'], widget=HBox…

It worked! Time to test the water isolation methods: 

In [9]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select("occurrence")
geom = feat.geometry()
img = ee.ImageCollection(img).map(sub.waterIsolationSWIR(gsw, geom))
Map.addLayer(img, viz, "Water", True)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=False)
Map

Map(center=[43.601748078270504, -73.57973412405578], controls=(WidgetControl(options=['position'], widget=HBox…